In [1]:
import os

# Directory containing images
directory = 'data\coco'

# Get list of files in the directory
files = os.listdir(directory)

# Iterate through files and remove those with extensions other than .jpeg, .jpg, and .png
for file in files:
    if not file.lower().endswith(('.jpeg', '.jpg', '.png')):
        os.remove(os.path.join(directory, file))
        print(f"Removed: {file}")


In [2]:
import tensorflow as tf
import os

In [3]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
tf.config.list_physical_devices('GPU')

[]

In [5]:


def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [6]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import Adam


# model
model = Sequential()

# first conv layer
# input shape = (img_rows, img_cols, 1)
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(100,120, 1))) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# second conv layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# flatten and put a fully connected layer
model.add(Flatten())
model.add(Dense(128, activation='relu')) # fully connected
model.add(Dropout(0.5))

# softmax layer
model.add(Dense(6, activation='softmax'))

# model summary
optimiser = Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 118, 32)       320       
                                                                 
 batch_normalization (Batch  (None, 98, 118, 32)       128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 59, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 49, 59, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 57, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 47, 57, 64)       

In [7]:

DATASET_PATH = 'data\coco'

dataset_path = os.path.join(DATASET_PATH, '*')
import glob
dataset_path = glob.glob(dataset_path)
dataset_path



['data\\coco\\0025.jpeg',
 'data\\coco\\0028.jpeg',
 'data\\coco\\0036.jpeg',
 'data\\coco\\0041.jpeg',
 'data\\coco\\0044.jpeg',
 'data\\coco\\0056.jpeg',
 'data\\coco\\0058.jpeg',
 'data\\coco\\0060.jpeg',
 'data\\coco\\0063.jpeg',
 'data\\coco\\0073.jpeg',
 'data\\coco\\0084.jpeg',
 'data\\coco\\0090.jpeg',
 'data\\coco\\0115.jpeg',
 'data\\coco\\0117.jpeg',
 'data\\coco\\0127.jpeg',
 'data\\coco\\0130.jpeg',
 'data\\coco\\0152.jpeg',
 'data\\coco\\0157.jpeg',
 'data\\coco\\0160.jpeg',
 'data\\coco\\0165.jpeg',
 'data\\coco\\0167.jpeg',
 'data\\coco\\0169.png',
 'data\\coco\\0175.jpeg',
 'data\\coco\\0180.jpeg',
 'data\\coco\\0194.jpeg',
 'data\\coco\\0196.jpeg',
 'data\\coco\\0206.jpeg',
 'data\\coco\\0212.jpeg',
 'data\\coco\\0225.jpeg',
 'data\\coco\\0228.jpeg',
 'data\\coco\\0231.jpeg',
 'data\\coco\\0233.jpeg',
 'data\\coco\\0250.jpeg',
 'data\\coco\\0252.jpeg',
 'data\\coco\\0255.jpeg',
 'data\\coco\\0262.jpeg',
 'data\\coco\\0268.jpeg',
 'data\\coco\\0272.jpeg',
 'data\\coco\

In [9]:
import cv2

# Load the image using cv2.imread()
image = cv2.imread(r'data\coco\0025.jpeg')

# Check if the image is loaded successfully
if image is not None:
    # Resize the image
    resized_image = cv2.resize(image, (100, 120))
    # Display the resized image
    cv2.imshow('Resized Image', resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print('Failed to load the image.')


In [10]:
import os
import cv2
import glob
import numpy as np

loaded_images = []
list_of_gestures = ['coco']

for gesture in list_of_gestures:
    dataset_path = "data//" + gesture
    gesture_path = os.path.join(dataset_path, '*')
    gest_path = glob.glob(gesture_path)
    k = 0
    for i in range(len(gest_path)):
        if k < 1600:
            image = cv2.imread(gest_path[i])
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray_image = cv2.resize(gray_image, (100, 120))
            loaded_images.append(gray_image)
        k += 1

print(len(loaded_images))

num_classes = 293
outputVectors = []

for i in range(len(loaded_images)):
    # Assuming your output vector needs to be all zeros except for one element set to 1
    output_vector = np.zeros(num_classes)
    output_vector[i % num_classes] = 1
    outputVectors.append(output_vector)

outputVectors = np.array(outputVectors)

print(outputVectors.shape)







67
(67, 293)


In [11]:
X = np.asarray(loaded_images)
y = np.asarray(outputVectors)
print(X.shape)
print(y.shape)




from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
X_train = X_train.reshape(X_train.shape[0], 100, 120, 1)
X_test = X_test.reshape(X_test.shape[0], 100, 120, 1)
print(X_train.shape)
print(X_test.shape)

(67, 120, 100)
(67, 293)
(53, 100, 120, 1)
(14, 100, 120, 1)
